In [ ]:
import torch

#remove the compute edge
def preProcessGraph(G):
  m,n=G.size()
  for i in range(m):
    for j in range(i,n):
      if G[i,j]>0 and G[j,i]>0:
        G[j,i]=0
  return G

# make the Graph be an upper triangle graph,
def preProcessDigGraph(G):
   m,n=G.size()
   for i in range(m):
    for j in range(i,n):
      G[j,i]=0
   return G

def findStart(G):
  m,n=G.size()
  result=torch.ones(m)
  for i in range(m):
    tmp=G[:,i]
    tmp1=tmp.sum()
    if tmp1>0:
      result[i]=0
  return result

def testG():
  G=torch.tensor([[1,0,1],[1,1,1],[1,1,1]])
  print(G)
  P=preProcessGraph(G)
  print(P)

  re1=findStart(P)
  print(re1)

  re2=preProcessDigGraph(G)
  print(re2)
testG() 

tensor([[1, 0, 1],
        [1, 1, 1],
        [1, 1, 1]])
tensor([[0, 0, 1],
        [1, 0, 1],
        [0, 0, 0]])
tensor([0., 1., 0.])
tensor([[0, 0, 1],
        [0, 0, 1],
        [0, 0, 0]])


 1. weight: every edge has two weight, [w,b], w is N×N, b is also N×N
 2. every node has 8 weights, μ, σ, A,w,phi,b k*N(μ, σ)+A sin(wt+Φ)+b; the demision is N×8
 3. the output is y= ∑k*𝐺(t,μ, σ)+Asin(wt+Φ)+b
 4. loss=mse||y-input||^2

 


In [ ]:
from torch.autograd.grad_mode import no_grad
from math import exp
import torch

data=torch.tensor([1.,4.,3.,2.,4.,3.],requires_grad=False)
data_size=data.size()
# print(data_size[0])

node_size=6
#generate a graph
graph=torch.ones(node_size,node_size)

#processing the graph
post_graph=preProcessDigGraph(graph)

print(post_graph)

#input should all be ones, dimension is node size
input=torch.ones(node_size)

#the output is data size
output=torch.zeros(node_size,data_size[0])
finalout=torch.zeros(data_size[0])
# nodeOut=torch.zeros(node_size,requires_grad=True)
nodeEnergy=torch.ones(node_size)

#node weight and bias
weight=torch.rand(node_size,node_size,requires_grad=True)
bias=torch.rand(node_size,node_size,requires_grad=True)

#output param
mu=torch.rand(node_size)
sigma=torch.rand(node_size)
K=torch.rand(node_size)
A=torch.rand(node_size)
omega=torch.rand(node_size)
phi=torch.rand(node_size)
outBias=torch.rand(node_size)

weight_mu=torch.rand(node_size,requires_grad=True)
bias_mu=torch.rand(node_size,requires_grad=True)

weight_sigma=torch.rand(node_size,requires_grad=True)
bias_sigma=torch.rand(node_size,requires_grad=True)

weight_K=torch.rand(node_size,requires_grad=True)
bias_K=torch.rand(node_size,requires_grad=True)

weight_A=torch.rand(node_size,requires_grad=True)
bias_A=torch.rand(node_size,requires_grad=True)

weight_omega=torch.rand(node_size,requires_grad=True)
bias_omega=torch.rand(node_size,requires_grad=True)

weight_phi=torch.rand(node_size,requires_grad=True)
bias_phi=torch.rand(node_size,requires_grad=True)

weight_outBias=torch.rand(node_size,requires_grad=True)
bias_outBias=torch.rand(node_size,requires_grad=True)
nodeOut=torch.rand(node_size,requires_grad=False)
# torch.autograd.set_detect_anomaly(True)

learning_rate=torch.tensor([0.001])
for iter in range (100):
   # only upper diagnal be processed:
    for i in range(node_size):
      # node energy
      nodeOut[i]=weight[i,i]*nodeEnergy[i]+outBias[i]
      nodeOut[i]=torch.relu(nodeOut[i])
      # tmp=nodeOut.clone()
      # for j in range(i,node_size):
      #   with torch.no_grad():
          # nodeEnergy[j]=nodeEnergy[j]+post_graph[i,j]*(nodeOut[i]*weight[i,j]+bias[i,j])

    # for i in range(node_size):
    #   mu[i]=weight_mu[i]*nodeOut[i]+bias_mu[i]
    #   sigma[i]=weight_sigma[i]*nodeOut[i]+bias_sigma[i]
    #   K[i]=weight_K[i]*nodeOut[i]+bias_K[i]
    #   A[i]=weight_A[i]*nodeOut[i]+bias_A[i]
    #   omega[i]=weight_omega[i]*nodeOut[i]+bias_omega[i]
    #   phi[i]=weight_phi[i]*nodeOut[i]+bias_phi[i]
    #   outBias[i]=weight_outBias[i]*nodeOut[i]+bias_outBias[i]

    # mu.retains_grad  

    for i in range(node_size):
      # for t in range(data_size[0]):
        finalout[i]=weight_mu[i]+nodeOut[i]+bias_mu[i]
        # output[i,t]=(K[i]/sigma[i]*torch.exp(-(t-mu[i]).pow(2)/(sigma[i].pow(2)))+A[i]*torch.sin(omega[i]*t+phi[i])+outBias[i])

    #get the loss:
    # finalout=torch.sum(output,0)


    loss=(finalout-data).pow(2).sum()
    
    loss.backward(retain_graph=True)
    print (loss)
    # print(finalout)

    #update the parameter:
    with torch.no_grad():
      weight-=learning_rate*weight.grad
    #   # bias-=learning_rate*bias.grad

      weight_mu-=learning_rate*weight_mu.grad
    #   weight_sigma-=learning_rate*weight_sigma.grad
    #   weight_K-=learning_rate*weight_K.grad
    #   weight_A-=learning_rate*weight_A.grad
    #   weight_omega-=learning_rate*weight_omega.grad
    #   weight_outBias-=learning_rate*weight_outBias.grad
    #   weight_phi-=learning_rate*weight_phi.grad

    #   bias_mu-=learning_rate*bias_mu.grad
    #   bias_sigma-=learning_rate*bias_sigma.grad
    #   bias_K-=learning_rate*bias_K.grad
    #   bias_A-=learning_rate*bias_A.grad
    #   bias_omega-=learning_rate*bias_omega.grad
    #   bias_outBias-=learning_rate*bias_outBias.grad
    #   bias_phi-=learning_rate*bias_phi.grad
     


tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor(13.5323, grad_fn=<SumBackward0>)
tensor(13.4242, grad_fn=<SumBackward0>)
tensor(13.2099, grad_fn=<SumBackward0>)
tensor(12.8926, grad_fn=<SumBackward0>)
tensor(12.4776, grad_fn=<SumBackward0>)
tensor(11.9713, grad_fn=<SumBackward0>)
tensor(11.3820, grad_fn=<SumBackward0>)
tensor(10.7190, grad_fn=<SumBackward0>)
tensor(9.9929, grad_fn=<SumBackward0>)
tensor(9.2153, grad_fn=<SumBackward0>)
tensor(8.3987, grad_fn=<SumBackward0>)
tensor(7.5562, grad_fn=<SumBackward0>)
tensor(6.7011, grad_fn=<SumBackward0>)
tensor(5.8471, grad_fn=<SumBackward0>)
tensor(5.0080, grad_fn=<SumBackward0>)
tensor(4.1970, grad_fn=<SumBackward0>)
tensor(3.4273, grad_fn=<SumBackward0>)
tensor(2.7110, grad_fn=<SumBackward0>)
tensor(2.0596, grad_fn=<SumBackward0>)
tensor(1.4836, grad_fn=<SumBackward0>)
tensor(

In [ ]:
print(output)
print(data)
print(loss)
print(finalout)

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])
tensor([1., 4., 3., 2., 4., 3.], requires_grad=True)
tensor(0.0500, grad_fn=<SumBackward0>)
tensor([1.0135, 3.8764, 2.9457, 2.0061, 3.8579, 2.8933], grad_fn=<CopySlices>)


In [ ]:
import torch

data=torch.tensor([1.,4.,3.,2.,4.,3.],requires_grad=True)

node_size=5
#generate a graph
graph=torch.ones(node_size,node_size)

#processing the graph
post_graph=preProcessDigGraph(graph)

print(post_graph)

input1=torch.tensor([1.,1.,1.,1.,1.,1.],requires_grad=True)

param_w=torch.rand(6,6,requires_grad=True)
param_b=torch.rand(6,6,requires_grad=True)
size=5

epson=torch.tensor([0.0001])

for j in range (2):
  out=torch.zeros(6)
  for i in range(6):
     out[i]=input1[i]*param_w[1,i]+param_b[1,i]
  #
  out1=torch.zeros(6)
  for i in range(6):
    out1[i]+=param_w[3,i]*torch.relu(param_w[2,i]*out[i])+param_w[4,i]
    
  loss=(out1-data).pow(2).sum()
  loss.backward()   
  # print(param_w.grad)

  with torch.no_grad():
    param_w-=epson*param_w.grad
    param_b-=epson*param_b.grad
  print(loss) 


tensor([[0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]])
tensor(38.4037, grad_fn=<SumBackward0>)
tensor(38.3811, grad_fn=<SumBackward0>)


In [ ]:
import torch
a=torch.rand(3,4)
c=torch.ones(3,4)
d=3/a
b=torch.sum(a)
print(b)
print(a)
print(d)

tensor(5.3962)
tensor([[0.3919, 0.5140, 0.5543, 0.2181],
        [0.5304, 0.0044, 0.6978, 0.6246],
        [0.5244, 0.5378, 0.0690, 0.7295]])
tensor([[  7.6557,   5.8368,   5.4118,  13.7565],
        [  5.6558, 685.0640,   4.2991,   4.8031],
        [  5.7209,   5.5784,  43.4747,   4.1123]])
